In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# taken from stackoverflow
def fibonacci_sphere(samples=100):

    points = []
    phi = math.pi * (3. - math.sqrt(5.))  # golden angle in radians

    for i in range(samples):
        y = 1 - (i / float(samples - 1)) * 2  # y goes from 1 to -1
        radius = math.sqrt(1 - y * y)  # radius at y

        theta = phi * i  # golden angle increment

        x = math.cos(theta) * radius
        z = math.sin(theta) * radius

        points.append((x, y, z))

    return points

sph = fibonacci_sphere(samples=50)

In [3]:
#Read all the data files
# Distance = 0.0 mm
d00 = pd.read_csv('d0.csv')
d00 = d00.rename(columns={"x": "Time", "Curve1": "Temperature"}, errors="raise")
d00['Time'] = d00['Time']/1000 #convert time column to picosecond

# Distance = 0.5 mm
d05 = pd.read_csv('d05.csv')
d05 = d05.rename(columns={"x": "Time", "Temp": "Temperature"}, errors="raise")
d05['Time'] = d05['Time']/1000 #convert time column to picosecond

# Distance = 1.0 mm
d10 = pd.read_csv('d1.csv')
d10 = d10.rename(columns={"x": "Time", "Curve1": "Temperature"}, errors="raise")
d10['Time'] = d10['Time']/1000 #convert time column to picosecond

# Distance = 1.5 mm
d15 = pd.read_csv('d15.csv')
d15 = d15.rename(columns={"x": "Time", "Curve1": "Temperature"}, errors="raise")
d15['Time'] = d15['Time']/1000 #convert time column to picosecond

In [4]:
# Choose time times to make interpolation
#time=np.arange(0,4.1,0.1) # in picosecond
time1 = np.array([0., 0.1])
time2 = np.arange(0.2,4.1,0.2)
time = np.concatenate((time1,time2))

# Make dataframe to each dataframe for data interpolation
d = {'Time': time, 'Temperature': np.nan}
dfAdd = pd.DataFrame(data=d)
dfAdd['Temperature'][0] = 0.

In [5]:
# Add dfAdd to the original dataframe
df00 = pd.concat([d00, dfAdd]).sort_values(by=['Time'])

# Interpolate
df00 = df00.interpolate()

# Choose only the rows that is in time
df00 = df00[df00['Time'].isin(time)]

# Add the x, y, z coordinates
df00['x'] = 0.0
df00['y'] = 0.0
df00['z'] = 0.0

f_df00 = df00[['x','y','z','Time','Temperature']]

In [6]:
fac = 500

# Add dfAdd to the original dataframe
df05 = pd.concat([d05, dfAdd]).sort_values(by=['Time'])

# Interpolate
df05 = df05.interpolate()

# Choose only the rows that is in time
df05 = df05[df05['Time'].isin(time)]

# Add the x, y, z coordinates
#df05['x'] = 0.5*1000
#df05['y'] = 0.0
#df05['z'] = 0.0

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df05['Time']):
    time_temp.append((df05['Time'][i],df05['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac) # Multiply by 500 as the coordinate is for a unit circle
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df500 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [7]:
fac = 1000

# Add dfAdd to the original dataframe
df10 = pd.concat([d10, dfAdd]).sort_values(by=['Time'])

# Interpolate
df10 = df10.interpolate()

# Choose only the rows that is in time
df10 = df10[df10['Time'].isin(time)]

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df10['Time']):
    time_temp.append((df10['Time'][i],df10['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df1000 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [8]:
fac = 1500

# Add dfAdd to the original dataframe
df15 = pd.concat([d15, dfAdd]).sort_values(by=['Time'])

# Interpolate
df15 = df15.interpolate()

# Choose only the rows that is in time
df15 = df15[df15['Time'].isin(time)]

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df15['Time']):
    time_temp.append((df15['Time'][i],df15['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df1500 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [9]:
# Make a dataframe for all known time, temperature and dist

df00t = df00.drop(columns=['x', 'y','z'])
df00t['Dist'] = 0.

df05t = df05
df05t['Dist'] = 500.

df10t = df10
df10t['Dist'] = 1000.

df15t = df15
df15t['Dist'] = 1500.
df15t

dft = pd.concat([df00t, df05t, df10t, df15t])

In [10]:
# Interpolate the dataframe to all the necessary dist values at each time steps
time = set(dft['Time'])
df_big = pd.DataFrame(columns=['Time', 'Temperature','Dist'])
dist = np.array([100.,200.,300.,400.,600.,700.,800.,900.,1100.,1200.,1300.,1400.])
for t in time:
    df_old = dft[dft['Time']==t]
    #print(df_old)
    d = {'Dist': dist, 'Temperature': np.nan, 'Time': t}
    dfAdd = pd.DataFrame(data=d)
    df_empty = pd.concat([df_old, dfAdd]).sort_values(by=['Dist'])
    df_fill = df_empty.interpolate()
    
    df_big = pd.concat([df_big, df_fill]).sort_values(by=['Time','Dist'])

/Users/suman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.
/Users/suman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


In [11]:
fac = 100

# Choose the data for given distance
df100 = df_big[df_big['Dist']==100].reset_index()

df100 = df100.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df100['Time']):
    time_temp.append((df100['Time'][i],df100['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df100 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [12]:
fac = 200

# Choose the data for given distance
df200 = df_big[df_big['Dist']==200].reset_index()

df200 = df200.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df200['Time']):
    time_temp.append((df200['Time'][i],df200['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df200 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [13]:
fac = 300

# Choose the data for given distance
df300 = df_big[df_big['Dist']==300].reset_index()

df300 = df300.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df300['Time']):
    time_temp.append((df300['Time'][i],df300['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df300 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [14]:
fac = 400

# Choose the data for given distance
df400 = df_big[df_big['Dist']==400].reset_index()

df400 = df400.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df400['Time']):
    time_temp.append((df400['Time'][i],df400['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df400 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [15]:
fac = 600

# Choose the data for given distance
df600 = df_big[df_big['Dist']==600].reset_index()

df600 = df600.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df600['Time']):
    time_temp.append((df600['Time'][i],df600['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

sph = fibonacci_sphere(samples=100)

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df600 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [16]:
fac = 700

# Choose the data for given distance
df700 = df_big[df_big['Dist']==700].reset_index()

df700 = df700.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df700['Time']):
    time_temp.append((df700['Time'][i],df700['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

sph = fibonacci_sphere(samples=100)

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df700 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [17]:
fac = 800

# Choose the data for given distance
df800 = df_big[df_big['Dist']==800].reset_index()

df800 = df800.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df800['Time']):
    time_temp.append((df800['Time'][i],df800['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

sph = fibonacci_sphere(samples=100)

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df800 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [18]:
fac = 900

# Choose the data for given distance
df900 = df_big[df_big['Dist']==900].reset_index()

df900 = df900.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df900['Time']):
    time_temp.append((df900['Time'][i],df900['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

sph = fibonacci_sphere(samples=150)

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df900 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [19]:
fac = 1000

# Choose the data for given distance
df1000 = df_big[df_big['Dist']==1000].reset_index()

df1000 = df1000.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df1000['Time']):
    time_temp.append((df1000['Time'][i],df1000['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

sph = fibonacci_sphere(samples=150)

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df1000 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [20]:
fac = 1100

# Choose the data for given distance
df1100 = df_big[df_big['Dist']==1100].reset_index()

df1100 = df1100.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df1100['Time']):
    time_temp.append((df1100['Time'][i],df1100['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

sph = fibonacci_sphere(samples=150)

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df1100 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [21]:
fac = 1200

# Choose the data for given distance
df1200 = df_big[df_big['Dist']==1200].reset_index()

df1200 = df1200.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df1200['Time']):
    time_temp.append((df1200['Time'][i],df1200['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

sph = fibonacci_sphere(samples=200)

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df1200 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [22]:
fac = 1300

# Choose the data for given distance
df1300 = df_big[df_big['Dist']==1300].reset_index()

df1300 = df1300.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df1300['Time']):
    time_temp.append((df1300['Time'][i],df1300['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

sph = fibonacci_sphere(samples=200)

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df1300 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [23]:
fac = 1400

# Choose the data for given distance
df1400 = df_big[df_big['Dist']==1400].reset_index()

df1400 = df1400.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df1400['Time']):
    time_temp.append((df1400['Time'][i],df1400['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

sph = fibonacci_sphere(samples=250)

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df1400 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [24]:
fac = 1500

# Choose the data for given distance
df1500 = df_big[df_big['Dist']==1500].reset_index()

df1500 = df1500.drop(columns=['Dist'])

# Append these time and temperature as tuple
time_temp = []
for i, row in enumerate(df1500['Time']):
    time_temp.append((df1500['Time'][i],df1500['Temperature'][i]))

# Create a array of x, y, z, time and temperature data 
x = []
y = []
z = []
time = []
temp = []

sph = fibonacci_sphere(samples=300)

for coordinates in sph:
    for val in time_temp:
        x.append(coordinates[0]*fac)
        y.append(coordinates[1]*fac)
        z.append(coordinates[2]*fac)
        time.append(val[0])
        temp.append(val[1])

# Put these arrays together to get the final dataframe
f_df1500 = pd.DataFrame(
    {'x': x,
     'y': y,
     'z': z,
     'Time': time,
     'Temperature': temp
    })

In [25]:
df_final = pd.concat([f_df00,f_df100,f_df200,f_df300,f_df400,f_df500,f_df600,f_df700,f_df800,
                     f_df900,f_df1000,f_df1100,f_df1200,f_df1300,f_df1400,f_df1500])

# Get rid of rows with time == 0
df_final = df_final[df_final['Time']!=0.]

In [26]:
df_final['Temperature'].max()

61268.5

In [27]:
df_final.to_csv('PlasmaTemp.csv')

In [28]:
# Create the dataframe for 2D scatter plot
# Data for distance 0
s_df00 = f_df00[['Time','Temperature']]
s_df00['Dist'] = 0.

# Data for distance 500
ff_df500 = f_df500[f_df500['y']==500.]
s_df500 = ff_df500[['Time','Temperature']]
s_df500['Dist'] = 500.

# Data for distance 1500.0
ff_df1500 = f_df1500[f_df1500['y']==1500.]
s_df1500 = ff_df1500[['Time','Temperature']]
s_df1500['Dist'] = 1500.

# Combine
s_df = pd.concat([s_df00, s_df500, s_df1500])

# Get rid of rows with time == 0
s_df = s_df[s_df['Time']!=0.]

/Users/suman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/suman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/suman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [29]:
s_df.to_csv('PlasmaPlot.csv')

In [30]:
# Save only one distance data
#dff=s_df00[['Time','Temperature']]
#dfff = dff[dff['Time']!=0.]
#dfff.to_csv('PlasmaPlot1.csv')